TensorFlow 模型导出 
使用 SavedModel 完整导出模型 
在部署模型时，我们的第一步往往是将训练好的整个模型完整导出为一系列标准格式的文件，然后即可在不同的平台上部署模型文件。这时，TensorFlow 为我们提供了 SavedModel 这一格式。与前面介绍的 Checkpoint 不同，SavedModel 包含了一个 TensorFlow 程序的完整信息： 不仅包含参数的权值，还包含计算的流程（即计算图） 。当模型导出为 SavedModel 文件时，无需建立模型的源代码即可再次运行模型，这使得 SavedModel 尤其适用于模型的分享和部署。后文的 TensorFlow Serving（服务器端部署模型）、TensorFlow Lite（移动端部署模型）以及 TensorFlow.js 都会用到这一格式。

Keras 模型均可方便地导出为 SavedModel 格式。不过需要注意的是，因为 SavedModel 基于计算图，所以对于使用继承 tf.keras.Model 类建立的 Keras 模型，其需要导出到 SavedModel 格式的方法（比如 call ）都需要使用 @tf.function 修饰（ @tf.function 的使用方式见 前文 ）。然后，假设我们有一个名为 model 的 Keras 模型，使用下面的代码即可将模型导出为 SavedModel：

tf.saved_model.save(model, "保存的目标文件夹名称")
在需要载入 SavedModel 文件时，使用

model = tf.saved_model.load("保存的目标文件夹名称")
即可。

提示

对于使用继承 tf.keras.Model 类建立的 Keras 模型 model ，使用 SavedModel 载入后将无法使用 model() 直接进行推断，而需要使用 model.call() 。

以下是一个简单的示例，将 前文 MNIST 手写体识别的模型 进行导出和导入。

导出模型到 saved/1 文件夹：

In [2]:
import tensorflow as t

# 数据获取及预处理
import numpy as np
class MNISTLoader():
    def __init__(self):
        mnist = tf.keras.datasets.mnist
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # MNIST中的图像默认为uint8（0-255的数字）。以下代码将其归一化到0-1之间的浮点数，并在最后增加一维作为颜色通道
        self.train_data = np.expand_dims(self.train_data.astype(np.float32) / 255.0, axis=-1)      # [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data.astype(np.float32) / 255.0, axis=-1)        # [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # [60000]
        self.test_label = self.test_label.astype(np.int32)      # [10000]
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # 从数据集中随机取出batch_size个元素并返回
        index = np.random.randint(0, np.shape(self.train_data)[0], batch_size)
        return self.train_data[index, :], self.train_label[index]

In [3]:
num_epochs = 1
batch_size = 50
learning_rate = 0.001

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
])

In [4]:
data_loader = MNISTLoader()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

In [5]:
model.fit(data_loader.train_data, data_loader.train_label, epochs=num_epochs, batch_size=batch_size)

Train on 60000 samples
60000/60000 [==============================] - 3s 43us/sample - loss: 0.2982 - sparse_categorical_accuracy: 0.9162


In [6]:
tf.saved_model.save(model, "saved/1")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved/1/assets


将 saved/1 中的模型导入并测试性能：

In [7]:
batch_size = 50

model = tf.saved_model.load("saved/1")
data_loader = MNISTLoader()
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)
for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    y_pred = model(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.951100


使用继承 tf.keras.Model 类建立的 Keras 模型同样可以以相同方法导出，唯须注意 call 方法需要以 @tf.function 修饰，以转化为 SavedModel 支持的计算图，代码如下：

In [8]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=10)

    @tf.function
    def call(self, inputs):         # [batch_size, 28, 28, 1]
        x = self.flatten(inputs)    # [batch_size, 784]
        x = self.dense1(x)          # [batch_size, 100]
        x = self.dense2(x)          # [batch_size, 10]
        output = tf.nn.softmax(x)
        return output

In [10]:
num_epochs = 1
batch_size = 50
learning_rate = 0.001

model = MLP()

data_loader = MNISTLoader()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

In [11]:
model.fit(data_loader.train_data, data_loader.train_label, epochs=num_epochs, batch_size=batch_size)

Train on 60000 samples
60000/60000 [==============================] - 3s 45us/sample - loss: 0.2959 - sparse_categorical_accuracy: 0.9168


In [12]:
tf.saved_model.save(model, "saved/2")

INFO:tensorflow:Assets written to: saved/2/assets


In [13]:
batch_size = 50

model = tf.saved_model.load("saved/2")
data_loader = MNISTLoader()
sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
num_batches = int(data_loader.num_test_data // batch_size)

for batch_index in range(num_batches):
    start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
    # 唯一不同的是需要显式调用call函数
    y_pred = model.call(data_loader.test_data[start_index: end_index])
    sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
print("test accuracy: %f" % sparse_categorical_accuracy.result())

test accuracy: 0.954600


Keras Sequential save 方法（Jinpeng）
我们以 keras 模型训练和保存为例进行讲解，如下是 keras 官方的 mnist 模型训练样例。

源码地址:

https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
以上代码，是基于 keras 的 Sequential 构建了多层的卷积神经网络，并进行训练。

为了方便起见可使用如下命令拷贝到本地:

curl -LO https://raw.githubusercontent.com/keras-team/keras/master/examples/mnist_cnn.py
然后，在最后加上如下一行代码（主要是对 keras 训练完毕的模型进行保存）:

model.save('mnist_cnn.h5')
在终端中执行 mnist_cnn.py 文件，如下:

python mnist_cnn.py
警告

该过程需要连接网络获取 mnist.npz 文件（https://s3.amazonaws.com/img-datasets/mnist.npz），会被保存到 $HOME/.keras/datasets/ 。如果网络连接存在问题，可以通过其他方式获取 mnist.npz 后，直接保存到该目录即可。

执行过程会比较久，执行结束后，会在当前目录产生 mnist_cnn.h5 文件（HDF5 格式），就是 keras 训练后的模型，其中已经包含了训练后的模型结构和权重等信息。

在服务器端，可以直接通过 keras.models.load_model("mnist_cnn.h5") 加载，然后进行推理；在移动设备需要将 HDF5 模型文件转换为 TensorFlow Lite 的格式，然后通过相应平台的 Interpreter 加载，然后进行推理。